In [1]:
import sys
sys.path.insert(1, "../gapfilling/scripts/")

In [2]:
# loading gap filling module
from Gapfilling import *
from modelseed_gapfilling import *
import random

In [3]:
import psamm

In [4]:
def make_model_copies(model, n = 10):
    d = {}
    N = n + 1
    for i in range(1, N):
        d[str(model) + "_" + str(i)] = fake_copy(model)
    return d

In [5]:
def remove_random_reaction(reference, model_dict, n = 1):
    """
    model_dict: dictionary of samples
    n: number of reactions to be removed from each model
    """
    n_samples = len(model_dict)
    N = n * n_samples
    
    reactions = list(reference.reactions)
    Random = random.sample(reactions, N)
    random_reactions = [i.id for i in Random]
    
    print(random_reactions)
    
    if N == n_samples:
        for i in range(len(model_dict)):
            list(model_dict.values())[i].remove_reactions([random_reactions[i]])
    else:
        c = 0
        for i in range(len(model_dict)):
            for j in range(c, c + n):
                list(model_dict.values())[i].remove_reactions([random_reactions[j]])
                c += 1
    return model_dict

In [6]:
# loading models
e_coli = read_sbml_model("../experiment/models/references/iML1515.xml")
l_lactis = read_sbml_model("../experiment/models/references/iNF517.xml")
p_putida = read_sbml_model("../experiment/models/references/iJN1463.xml")

Using license file /home/fco/gurobi.lic
Academic license - for non-commercial use only - expires 2021-08-10


In [7]:
# Checking objective functions
print("OBJECTIVE FUNCTIONS:\n")
print("E. coli:\n\n" + str(e_coli.objective) + "\n")
print("L. lactis:\n\n" + str(l_lactis.objective) + "\n")
print("P. putida:\n\n" + str(p_putida.objective) + "\n")

OBJECTIVE FUNCTIONS:

E. coli:

Maximize
1.0*BIOMASS_Ec_iML1515_core_75p37M - 1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685

L. lactis:

Maximize
1.0*BIOMASS_LLA - 1.0*BIOMASS_LLA_reverse_0796e

P. putida:

Maximize
1.0*BIOMASS_KT2440_WT3 - 1.0*BIOMASS_KT2440_WT3_reverse_d86d5



All three models are meant to optimized biomass production. Let's change the objective function of one of them in order to have a more diverse dataset.

In [8]:
p_putida.reactions.ACS

Reaction identifier,ACS
Name,Acetyl-CoA synthetase
Memory address,0x07ff5a82163d0
Stoichiometry,ac_c + atp_c + coa_c --> accoa_c + amp_c + ppi_c Acetate + ATP C10H12N5O13P3 + Coenzyme A --> Acetyl-CoA + AMP C10H12N5O7P + Diphosphate
GPR,PP_4487 or PP_4702 or PP_3071
Lower bound,0.0
Upper bound,999999.0


In [9]:
p_putida.objective = "ACS"

In [10]:
# Checking the fluxes of each model
print("FLUX VALUES:\n")
print("E. coli:\n\n" + str(e_coli.optimize().objective_value) + "\n")
print("L. lactis:\n\n" + str(l_lactis.optimize().objective_value) + "\n")
print("P. putida:\n\n" + str(p_putida.optimize().objective_value) + "\n")

FLUX VALUES:

E. coli:

0.8769972144269736

L. lactis:

0.04263460544337326

P. putida:

129.57999999999998



In [13]:
# Creating 10 copies of each model

e_coli_samples = make_model_copies(e_coli)
l_lactis_samples = make_model_copies(l_lactis)
p_putida_samples = make_model_copies(p_putida)

Now, we will remove one random reaction of every copy. 

In [14]:
print(len(list(e_coli_samples.values())[5].reactions))
print(len(list(l_lactis_samples.values())[5].reactions))
print(len(list(p_putida_samples.values())[5].reactions))

2712
754
2927


In [15]:
# removing one reaction from every model

remove_random_reaction(e_coli, e_coli_samples)
remove_random_reaction(p_putida, p_putida_samples)
remove_random_reaction(l_lactis, l_lactis_samples)

['EX_g1p_e', 'MALTHXtexi', 'THRt2pp', 'EX_fru_e', '5DGLCNR', 'DHNCOAT', 'UDCPDPS', 'FRUtex', 'EX_chtbs_e', 'GLYCK2']
['GLYCAt2rpp', 'LTDCL', 'SADH', 'EX_34dhcinm_e', 'EX_so3_e', 'LPADSS2_1', '3OAS181', 'DMPPS', 'OMBZLM', 'ALGE8']
['OCMAT3', 'HPPK', 'EX_glcn__D_e', 'EX_pyr_e', 'ILETRS_1', 'AGAT_LLA', 'EX_ser__L_e', 'MTHFC', 'ALAR', 'PRAIS_1']


{'iNF517_1': <Model iNF517 at 0x7ff519f77130>,
 'iNF517_2': <Model iNF517 at 0x7ff517212bb0>,
 'iNF517_3': <Model iNF517 at 0x7ff516f5f460>,
 'iNF517_4': <Model iNF517 at 0x7ff5184d9ac0>,
 'iNF517_5': <Model iNF517 at 0x7ff5162c3460>,
 'iNF517_6': <Model iNF517 at 0x7ff515c561f0>,
 'iNF517_7': <Model iNF517 at 0x7ff5154ad6d0>,
 'iNF517_8': <Model iNF517 at 0x7ff514f231c0>,
 'iNF517_9': <Model iNF517 at 0x7ff514844580>,
 'iNF517_10': <Model iNF517 at 0x7ff5141d7e50>}

In [16]:
print(len(list(e_coli_samples.values())[5].reactions))
print(len(list(l_lactis_samples.values())[5].reactions))
print(len(list(p_putida_samples.values())[5].reactions))

2711
753
2926


In [17]:
# checking functionality of the models
# e. coli
for i in range(len(e_coli_samples)):
    print(list(e_coli_samples.keys())[i])
    print(list(e_coli_samples.values())[i].optimize().objective_value)

iML1515_1
0.8769972144269712
iML1515_2
0.8769972144269669
iML1515_3
0.8769972144269712
iML1515_4
0.8769972144269763
iML1515_5
0.8769972144269759
iML1515_6
0.876997214426967
iML1515_7
0.0
iML1515_8
0.8769972144269678
iML1515_9
0.8769972144269725
iML1515_10
0.8769972144269655


In [18]:
# l. lactis
for i in range(len(l_lactis_samples)):
    print(list(l_lactis_samples.keys())[i])
    print(list(l_lactis_samples.values())[i].optimize().objective_value)

iNF517_1
0.0
iNF517_2
0.0
iNF517_3
0.04263460544337327
iNF517_4
0.04263460544337326
iNF517_5
0.0
iNF517_6
0.0
iNF517_7
0.04263460544337325
iNF517_8
0.04263460544337326
iNF517_9
0.0
iNF517_10
0.04263460544337327


In [19]:
# p. putida
for i in range(len(p_putida_samples)):
    print(list(p_putida_samples.keys())[i])
    print(list(p_putida_samples.values())[i].optimize().objective_value)

iJN1463_1
129.57999999999998
iJN1463_2
129.57999999999998
iJN1463_3
129.57999999999998
iJN1463_4
129.58000000000004
iJN1463_5
129.58000000000004
iJN1463_6
129.58000000000004
iJN1463_7
129.58000000000004
iJN1463_8
129.57999999999998
iJN1463_9
129.57999999999998
iJN1463_10
129.57999999999998


In [30]:
# Make copies of each set of models for comparison
e_coli_cobra = {}
e_coli_gallant = {}
for x in e_coli_samples:
    e_coli_cobra[x] = fake_copy(e_coli_samples[x])
    e_coli_gallant[x] = fake_copy(e_coli_samples[x])

In [40]:
l_lactis_cobra = {}
l_lactis_gallant = {}
for x in l_lactis_samples:
    l_lactis_cobra[x] = fake_copy(l_lactis_samples[x])
    l_lactis_gallant[x] = fake_copy(l_lactis_samples[x])

In [32]:
p_putida_cobra = {}
p_putida_gallant = {}
for x in p_putida_samples:
    p_putida_cobra[x] = fake_copy(p_putida_samples[x])
    p_putida_gallant[x] = fake_copy(p_putida_samples[x])

In [33]:
%%timeit -n1 -r1
# E. coli
# COBRA
for x in e_coli_cobra:
    print(x)
    with suppress_stdout():
        sol = 0
        try:
            sol = cobra.flux_analysis.gapfill(e_coli_cobra[x], e_coli, demand_reactions=False)
        except exceptions.Infeasible:
            continue
        except RuntimeError:
            continue
    print(sol[0])

iML1515_1
[]
iML1515_2
[]
iML1515_3
[]
iML1515_4
[]
iML1515_5
[]
iML1515_6
[]
iML1515_7
iML1515_8
[]
iML1515_9
[]
iML1515_10
[]
2min 56s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [34]:
%%timeit -n1 -r1
# GALLANT
for x in e_coli_gallant:
    print(x)
    with suppress_stdout():
        sol = homology_gapfilling(e_coli_gallant[x], [e_coli], integer_threshold=1e-9,
                                  force_exchange=True, force_transport=True)
    print(sol[1])

iML1515_1
{'iML1515': [('EX_g1p_e', 'Exchange reaction')]}
iML1515_2
{'iML1515': [('MALTHXtexi', ['b4036'])]}
iML1515_3
{'iML1515': [('THRt2pp', ['b0813', 'b3823'])]}
iML1515_4
{'iML1515': [('EX_fru_e', 'Exchange reaction')]}
iML1515_5
{'iML1515': []}
iML1515_6
{'iML1515': []}
iML1515_7
{}
iML1515_8
{'iML1515': [('FRUtex', ['b1377', 'b0241', 'b0929', 'b2215'])]}
iML1515_9
{'iML1515': [('EX_chtbs_e', 'Exchange reaction')]}
iML1515_10
{'iML1515': []}
3min 29s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [41]:
%%timeit -n1 -r1
# L. lactis
# COBRA
for x in l_lactis_cobra:
    print(x)
    sol = 0
    with suppress_stdout():
        try:
            sol = cobra.flux_analysis.gapfill(l_lactis_cobra[x], l_lactis, demand_reactions=False)
        except exceptions.Infeasible:
            continue
        except RuntimeError:
            continue
    print(sol[0])

iNF517_1
iNF517_2
iNF517_3
iNF517_4
iNF517_5
iNF517_6
iNF517_7
iNF517_8
iNF517_9
iNF517_10
47.2 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [43]:
%%timeit -n1 -r1
# GALLANT
for x in l_lactis_gallant:
    print(x)
    with suppress_stdout():
        sol = homology_gapfilling(l_lactis_gallant[x], [l_lactis], integer_threshold=1e-9,
                                  force_exchange=True, force_transport=True)
    print(sol[1])

iNF517_1
{}
iNF517_2
{}
iNF517_3
{}
iNF517_4
{}
iNF517_5
{}
iNF517_6
{}
iNF517_7
{}
iNF517_8
{}
iNF517_9
{}
iNF517_10
{}
49.3 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [44]:
%%timeit -n1 -r1
# P. putida
# COBRA
for x in p_putida_cobra:
    print(x)
    sol = 0
    with suppress_stdout():
        try:
            sol = cobra.flux_analysis.gapfill(p_putida_cobra[x], p_putida, demand_reactions=False)
        except exceptions.Infeasible:
            continue
        except RuntimeError:
            continue
    print(sol[0])

iJN1463_1
[]
iJN1463_2
[]
iJN1463_3
[]
iJN1463_4
[]
iJN1463_5
[]
iJN1463_6
[]
iJN1463_7
[]
iJN1463_8
[]
iJN1463_9
[]
iJN1463_10
[]
3min 10s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [45]:
%%timeit -n1 -r1
# GALLANT
for x in p_putida_gallant:
    print(x)
    with suppress_stdout():
        sol = homology_gapfilling(p_putida_gallant[x], [p_putida], integer_threshold=1e-9,
                                  force_exchange=True, force_transport=True)
    print(sol[1])

iJN1463_1
{'iJN1463': [('GLYCAt2rpp', ['PP_3600'])]}
iJN1463_2
{'iJN1463': []}
iJN1463_3
{'iJN1463': []}
iJN1463_4
{'iJN1463': [('EX_34dhcinm_e', 'Exchange reaction')]}
iJN1463_5
{'iJN1463': [('EX_so3_e', 'Exchange reaction')]}
iJN1463_6
{'iJN1463': []}
iJN1463_7
{'iJN1463': []}
iJN1463_8
{'iJN1463': []}
iJN1463_9
{'iJN1463': []}
iJN1463_10
{'iJN1463': []}
3min 27s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [36]:
X = {}
for x in e_coli_samples:
    X[x] = fake_copy(e_coli_samples[x])